## BIL462 Project Source Codes | 2024-2025 Fall | Context: Iris Recognition on UBIRIS.v2 Dataset

#### Step1: Sentetic Noise Addition (Image Processing)

In [2]:
%pip install tqdm

import os
import cv2
import numpy as np 
from tqdm import tqdm

### Environment Setup and Library Imports

The following code sets up the environment and imports the necessary libraries for the project:

```python
%pip install tqdm

import os
import cv2
import numpy as np
from tqdm import tqdm
```

#### Explanation:

1. **`%pip install tqdm`**
   - A **magic command** in Jupyter Notebook to install the `tqdm` package directly within the notebook.
   - `tqdm` is a Python library used to add progress bars to loops, which helps in tracking time-consuming tasks like image processing or training.

2. **`import os`**
   - The `os` module provides a way to interact with the operating system, including handling file paths, directories, and system-level operations.

3. **`import cv2`**
   - `cv2` is part of OpenCV (Open Source Computer Vision Library).
   - It is used for image processing tasks, such as reading, writing, and manipulating images.

4. **`import numpy as np`**
   - `numpy` is a library for numerical operations on arrays and matrices, essential for handling image data, typically represented as arrays.

5. **`from tqdm import tqdm`**
   - Imports the `tqdm` function for displaying progress bars in loops, enhancing user experience by showing how far a process has progressed.

>The purpose of tqdm is to provide a fast, extensible progress bar for Python loops and operations. It allows users to visually track the progress of time-consuming tasks, making it especially helpful for tasks involving large datasets or long-running computations.

In [7]:

def add_motion_blur(image, kernel_size=15):
   
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    kernel = kernel / kernel_size
    blurred = cv2.filter2D(image, -1, kernel)
    return blurred


def apply_low_light(image, gamma=0.3):
    """
    Düşük ışık koşullarını simüle etmek için gamma düzeltmesi uygula.
    """
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype("uint8")
    low_light_image = cv2.LUT(image, table)
    return low_light_image

 
input_folder = "CLASSES_400_300"  # Orijinal görüntülerin bulunduğu klasör
output_folder = "CLASSES_400_300_noised"  # Gürültü eklenmiş görüntülerin kaydedileceği klasör

# Çıkış klasörünü oluştur
os.makedirs(output_folder, exist_ok=True)

In [12]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Initialize counters for different filters
motion_blur_count = 0
low_light_count = 0
both_count = 0

# Apply filters to all images in the input folder
for image_name in tqdm(os.listdir(input_folder)):
    image_path = os.path.join(input_folder, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        print(f"Error reading image {image_path}")
        continue

    noisy_image = image.copy()  # Create a copy of the image to apply filters

    # Randomly choose a filter type with given probabilities
    filters = np.random.choice(["motion_blur", "low_light", "both"], p=[0.4, 0.4, 0.2])

    if filters == "motion_blur":
        # Apply motion blur filter
        noisy_image = add_motion_blur(noisy_image, kernel_size=15)
        motion_blur_count += 1
    elif filters == "low_light":
        # Apply low-light filter
        noisy_image = apply_low_light(noisy_image, gamma=0.4)
        low_light_count += 1
    else:  # both
        # Apply both motion blur and low-light filters
        noisy_image = add_motion_blur(noisy_image, kernel_size=15)
        noisy_image = apply_low_light(noisy_image, gamma=0.4)
        both_count += 1

    # Save the filtered image to the output folder
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, noisy_image)

# Calculate the total number of images
total_images = len(os.listdir(input_folder))

# Print summary of the process
print("Noisy images created successfully.")
print("Total number of images:", total_images)
print("Output folder:", output_folder)
print("Filter application ratio (motion_blur):", motion_blur_count / total_images)
print("Filter application ratio (low_light):", low_light_count / total_images)
print("Filter application ratio (both):", both_count / total_images)


100%|██████████| 1/1 [00:00<00:00, 116.33it/s]

Noisy images created successfully.
Total number of images: 1
Output folder: CLASSES_400_300_noised
Filter application ratio (motion_blur): 0.0
Filter application ratio (low_light): 1.0
Filter application ratio (both): 0.0


#### Step2: Data Augmentation

In [14]:
import numpy as np
import glob
import cv2
import os

# Paths
input_folder = r"CLASSES_400_300"
output_folder = r"CLASSES_400_300_augmented"
# r before the string denotes a raw string literal in python
# it ensure that backslasher in the string are treate as literal charater
os.makedirs(output_folder, exist_ok=True)

# Functions
def rotate_image(image, label, number):
    rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    output_path = os.path.join(output_folder, f"{label}_rotate_{number}.jpg")
    cv2.imwrite(output_path, rotated_image)

def flip_image(image, direction, label, number):
    flipped_image = cv2.flip(image, direction)
    output_path = os.path.join(output_folder, f"{label}_flip_{direction}_{number}.jpg")
    cv2.imwrite(output_path, flipped_image)

def add_light(image, gamma, label, number):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    brightened_image = cv2.LUT(image, table)
    output_path = os.path.join(output_folder, f"{label}_light_{gamma}_{number}.jpg")
    cv2.imwrite(output_path, brightened_image)

# Data augmentation function
def augment_images(input_folder, percentage):
    """
    Augments images in the specified folder and generates new images based on the given percentage.
    Args:
        input_folder (str): Path to the folder containing input images.
        percentage (float): Percentage of new images to generate.
    Returns:
        None
    Process:
        - Retrieves all .tiff files in the input folder.
        - Calculates the target number of new images based on the total number of images.
        - Performs data augmentation operations on each image:
            - Rotates the image
            - Flips the image
            - Increases the brightness of the image
            - Decreases the brightness of the image
        - Stops once the specified percentage is reached.
        - Prints the number and percentage of generated images.
    """
    images = glob.glob(os.path.join(input_folder, "*.tiff"))
    total_images = len(images)
    target_count = int(total_images * (percentage / 100))  # Percentage target count
    count = 0

    for filepath in images:
        if count >= target_count:  # Stop if the target percentage is reached
            break

        filename = os.path.basename(filepath)
        image = cv2.imread(filepath)
        label = os.path.splitext(filename)[0]  # File name label

        # Data augmentation operations
        rotate_image(image, label, count)
        count += 1
        if count >= target_count: break

        flip_image(image, 0, label, count)
        count += 1
        if count >= target_count: break

        add_light(image, 1.5, label, count)
        count += 1
        if count >= target_count: break

        add_light(image, 0.7, label, count)
        count += 1
        if count >= target_count: break

    print(f"{count} images ({percentage}%) successfully generated and saved.")

# User-defined percentage N value
N = 20  # For example, generate 20% of the total number of images
augment_images(input_folder, N)


0 images (20%) successfully generated and saved.


#### Step3: Image Enhancement (DeepLearning)

In [ ]:
%pip install torch torchvision scikit-image

import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [5]:
# Dataset Tanımı
class NoiseReductionDataset(Dataset):
    def __init__(self, noisy_folder, original_folder, transform=None):
        self.noisy_images = sorted(os.listdir(noisy_folder))
        self.original_images = sorted(os.listdir(original_folder))
        self.noisy_folder = noisy_folder
        self.original_folder = original_folder
        self.transform = transform

    def __len__(self):
        return len(self.noisy_images)

    def __getitem__(self, idx):
        noisy_path = os.path.join(self.noisy_folder, self.noisy_images[idx])
        original_path = os.path.join(self.original_folder, self.original_images[idx])

        noisy_image = cv2.imread(noisy_path, cv2.IMREAD_COLOR)
        original_image = cv2.imread(original_path, cv2.IMREAD_COLOR)

        if self.transform:
            noisy_image = self.transform(noisy_image)
            original_image = self.transform(original_image)

        return noisy_image, original_image, os.path.basename(noisy_path)

# Dataset Dönüşümleri
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

# Dataset ve DataLoader
noisy_folder = "CLASSES_400_300_noised"
original_folder = "CLASSES_400_300"
dataset = NoiseReductionDataset(noisy_folder, original_folder, transform=transform)

In [11]:
# Verileri Train/Validation/Test Split Yapma
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

NameError: name 'dataset' is not defined

In [12]:
# DenseNet Encoder ve Karmaşık Decoder Modeli
class DenseNetModel(nn.Module):
    def __init__(self):
        super(DenseNetModel, self).__init__()
        self.encoder = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1).features
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 256, kernel_size=3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()  # Renk bozulmalarını azaltmak için
        )

    def forward(self, x):
        original_size = x.size()[2:]  # Giriş boyutlarını kaydet
        x = self.encoder(x)  # Encoder işlemi
        x = self.decoder(x)  # Decoder işlemi
        x = F.interpolate(x, size=original_size, mode="bilinear", align_corners=False)  # Giriş boyutlarına geri döndür
        return x

# Kayıp Fonksiyonu (MSE + SSIM)
class CombinedLoss(nn.Module):
    def __init__(self, data_range=1.0):
        super(CombinedLoss, self).__init__()
        self.mse = nn.MSELoss()
        self.data_range = data_range

    def forward(self, outputs, targets):
        # MSE kaybı
        mse_loss = self.mse(outputs, targets)
        
        # NumPy dizilerine dönüştürme
        outputs_np = outputs.detach().cpu().numpy()
        targets_np = targets.cpu().numpy()
        
        # SSIM hesabı (data_range parametresi eklendi)
        ssim_loss = np.mean([
            1 - ssim(
                targets_np[i].transpose(1, 2, 0),
                outputs_np[i].transpose(1, 2, 0),
                channel_axis=-1,
                data_range=self.data_range
            )
            for i in range(len(outputs_np))
        ])
        
        return mse_loss + 0.5 * ssim_loss

# PSNR Hesaplama Fonksiyonu
def calculate_psnr(mse):
    return 20 * np.log10(1.0 / np.sqrt(mse))


NameError: name 'nn' is not defined

In [13]:
# For using GPU - CUDA
# %pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Model, Loss Fonksiyonu ve Optimizasyon
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNetModel().to(device)
criterion = CombinedLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
print("Torch Version:", torch.__version__, ", CUDA Available:", torch.cuda.is_available())

# Eğitim ve Metrik Kayıtları
train_mse, val_mse = [], []
train_ssim, val_ssim = [], []
train_psnr, val_psnr = [], []

NameError: name 'torch' is not defined

In [14]:
# Eğitim Döngüsü
num_epochs = 20
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    epoch_train_mse, epoch_train_ssim, epoch_train_psnr = 0, 0, 0

    for noisy_images, original_images, _ in tqdm(train_loader):
        noisy_images = noisy_images.to(device)
        original_images = original_images.to(device)

        # Tahmin ve kayıp hesaplama
        outputs = model(noisy_images)
        loss = criterion(outputs, original_images)

        # Geri yayılım
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrik hesaplama
        epoch_train_mse += loss.item()
        outputs_np = outputs.detach().cpu().numpy()
        original_np = original_images.cpu().numpy()

        batch_ssim = np.mean([
            ssim(original_np[i].transpose(1, 2, 0),
                outputs_np[i].transpose(1, 2, 0),
                channel_axis=-1,
                data_range=1.0)
            for i in range(len(outputs_np))
        ])
        epoch_train_ssim += batch_ssim

        batch_psnr = calculate_psnr(loss.item())
        epoch_train_psnr += batch_psnr

    # Ortalama değerler
    train_mse.append(epoch_train_mse / len(train_loader))
    train_ssim.append(epoch_train_ssim / len(train_loader))
    train_psnr.append(epoch_train_psnr / len(train_loader))

    # Doğrulama Performansı
    model.eval()
    epoch_val_mse, epoch_val_ssim, epoch_val_psnr = 0, 0, 0
    with torch.no_grad():
        for noisy_images, original_images, _ in val_loader:
            noisy_images = noisy_images.to(device)
            original_images = original_images.to(device)

            outputs = model(noisy_images)
            loss = criterion(outputs, original_images)

            epoch_val_mse += loss.item()
            outputs_np = outputs.cpu().numpy()
            original_np = original_images.cpu().numpy()

            batch_ssim = np.mean([
                ssim(original_np[i].transpose(1, 2, 0),
                    outputs_np[i].transpose(1, 2, 0),
                    channel_axis=-1,
                    data_range=1.0)  # Make sure your data is indeed scaled to [0,1]
                for i in range(len(outputs_np))
            ])
            epoch_val_ssim += batch_ssim

            batch_psnr = calculate_psnr(loss.item())
            epoch_val_psnr += batch_psnr

    val_mse.append(epoch_val_mse / len(val_loader))
    val_ssim.append(epoch_val_ssim / len(val_loader))
    val_psnr.append(epoch_val_psnr / len(val_loader))

    scheduler.step(val_mse[-1])

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train MSE: {train_mse[-1]:.4f}, Val MSE: {val_mse[-1]:.4f}, "
          f"Train SSIM: {train_ssim[-1]:.4f}, Val SSIM: {val_ssim[-1]:.4f}, "
          f"Train PSNR: {train_psnr[-1]:.2f}, Val PSNR: {val_psnr[-1]:.2f}")

    if val_mse[-1] < best_val_loss:
        best_val_loss = val_mse[-1]
        torch.save(model.state_dict(), "best_densenet_model.pth")

NameError: name 'model' is not defined

In [15]:
# Test Performansı
model.load_state_dict(torch.load("best_densenet_model.pth", weights_only=True))
model.eval()
test_mse, test_ssim, test_psnr = 0, 0, 0

denoised_folder = "CLASSES_400_300_denoised"
os.makedirs(denoised_folder, exist_ok=True)

with torch.no_grad():
    for noisy_images, original_images, file_names in tqdm(test_loader):
        noisy_images = noisy_images.to(device)
        original_images = original_images.to(device)

        outputs = model(noisy_images)
        loss = criterion(outputs, original_images)

        test_mse += loss.item()
        outputs_np = outputs.cpu().numpy()
        original_np = original_images.cpu().numpy()

        batch_ssim = np.mean([
            ssim(original_np[i].transpose(1, 2, 0),
                outputs_np[i].transpose(1, 2, 0),
                channel_axis=-1,
                data_range=1.0)
            for i in range(len(outputs_np))
        ])
        test_ssim += batch_ssim

        batch_psnr = calculate_psnr(loss.item())
        test_psnr += batch_psnr

        for i in range(outputs_np.shape[0]):
            restored = np.clip((outputs_np[i].transpose(1, 2, 0) * 255), 0, 255).astype(np.uint8)
            file_name = file_names[i]
            output_path = os.path.join(denoised_folder, file_name)
            cv2.imwrite(output_path, restored)

test_mse /= len(test_loader)
test_ssim /= len(test_loader)
test_psnr /= len(test_loader)

print(f"Test MSE: {test_mse:.4f}, Test SSIM: {test_ssim:.4f}, Test PSNR: {test_psnr:.2f}")

NameError: name 'model' is not defined

In [16]:
# Kayıp ve Metrik Grafiklerini Çiz
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.plot(train_mse, label="Train MSE", marker="o")
plt.plot(val_mse, label="Validation MSE", marker="o")
plt.title("Train and Validation MSE")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend()
plt.grid()

plt.subplot(1, 3, 2)
plt.plot(train_ssim, label="Train SSIM", marker="o")
plt.plot(val_ssim, label="Validation SSIM", marker="o")
plt.title("Train and Validation SSIM")
plt.xlabel("Epoch")
plt.ylabel("SSIM")
plt.legend()
plt.grid()

plt.subplot(1, 3, 3)
plt.plot(train_psnr, label="Train PSNR", marker="o")
plt.plot(val_psnr, label="Validation PSNR", marker="o")
plt.title("Train and Validation PSNR")
plt.xlabel("Epoch")
plt.ylabel("PSNR (dB)")
plt.legend()
plt.grid()

plt.savefig("metrics_densenet.png")
plt.show()

NameError: name 'plt' is not defined